# Este es un ejemplo de CRFs para resolver la tarea de NER, usando la herramienta crfsuite  referenciado en la página https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb . Este modelo agrega más características al modelo baseline que son de forma (shape) de la palabras.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [2]:
from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

## Uso de CoNLL2002 para  reconocimiento de entidades +CRF

El corpis de CoNLL2002 tiene especificados los archivos de los conjuntos de entrenamiento, evaluación y testeo. 

In [3]:
nltk.corpus.conll2002.fileids()

['esp.testa', 'esp.testb', 'esp.train', 'ned.testa', 'ned.testb', 'ned.train']

In [4]:
%%time
train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

CPU times: user 1.52 s, sys: 69.4 ms, total: 1.59 s
Wall time: 1.6 s


In [5]:
train_sents[0]

[('Melbourne', 'NP', 'B-LOC'),
 ('(', 'Fpa', 'O'),
 ('Australia', 'NP', 'B-LOC'),
 (')', 'Fpt', 'O'),
 (',', 'Fc', 'O'),
 ('25', 'Z', 'O'),
 ('may', 'NC', 'O'),
 ('(', 'Fpa', 'O'),
 ('EFE', 'NC', 'B-ORG'),
 (')', 'Fpt', 'O'),
 ('.', 'Fp', 'O')]

## Selección de características

Se definen las caraterísticas que se adicionan al vector de entrenamiento X, sufijo de la palabra, forma de la palabra el pos tag  y  la información cercana a la palabra. Este modelo hace fácil la creación de otros modelos a partir del modelo base usando CRFs.

sklearn-crfsuite y python-crfsuite soporta varios formatos de características.

In [6]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],        
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True
                
    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

Se extraen las características para cada sentencia de entrenamiento, sent2features; extrae las características de forma de cada palabra, sent2labels, obtiene la etiqueta de la palabra y sent2tokens el token de la palabra.

In [7]:
print(sent2features(train_sents[0])[0])
print(sent2labels(train_sents[0])[0])
print(sent2tokens(train_sents[0])[0])

{'bias': 1.0, 'word.lower()': 'melbourne', 'word[-3:]': 'rne', 'word[-2:]': 'ne', 'word.isupper()': False, 'word.istitle()': True, 'word.isdigit()': False, 'postag': 'NP', 'postag[:2]': 'NP', 'BOS': True, '+1:word.lower()': '(', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'Fpa', '+1:postag[:2]': 'Fp'}
B-LOC
Melbourne


Se genera el conjunto de entrenamiento y el de testeo

In [8]:
%%time
X_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]
y_test = [sent2labels(s) for s in test_sents]

CPU times: user 825 ms, sys: 85.4 ms, total: 910 ms
Wall time: 918 ms


## Training

To see all possible CRF parameters check its docstring. Here we are useing L-BFGS training algorithm (it is default) with Elastic Net (L1 + L2) regularization.

In [9]:
%%time
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)
crf.fit(X_train, y_train)

CPU times: user 27.3 s, sys: 238 ms, total: 27.5 s
Wall time: 27.7 s


/Users/raulernestogutierrezdepinerezreyes/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=True,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

## Evaluation

There is much more O entities in data set, but we're more interested in other entities. To account for this we'll use averaged F1 score computed for all labels except for O. ``sklearn-crfsuite.metrics`` package provides some useful metrics for sequence classification task, including this one.

In [10]:

sentence2 = ['el', 'hombre', 'bajo','canta', 'bajó', 'el', 'Puente', 'rojo', 'en','Sao','Paulo','en','la','escalera', 
            'baja']
sentence=['Correr','en','Colombia','con', 'James', 'Rodriguez', 'Éste','gran', 'hombre', 'ganó', 
            'con', 'el', 'Real', 'Madrid','de','España', 'en','Alemania','con', 'Roler']
sentence1= ['Correr', 'en', 'Colombia', 'con', 'James', 'Rodriguez', 'Éste', 'gran', 'hombre', 'ganó', 
            'con', 'el', 'Real', 'Madrid', 'de', 'España', 'y', 'en', 'Alemania', 'con', 'Roler.']
           
def pos_tag(sentence):
    sentence_features = [word2features(sentence, index) for index in range(len(sentence))]
    return list(zip(sentence, crf.predict([sentence_features])[0]))
 
print(pos_tag(sentence))  # [('I', 'PRP'), ('am', 'VBP'), ('Bob', 'NNP'), ('!', '.')]

[('Correr', 'O'), ('en', 'O'), ('Colombia', 'B-ORG'), ('con', 'O'), ('James', 'B-ORG'), ('Rodriguez', 'I-ORG'), ('Éste', 'I-ORG'), ('gran', 'O'), ('hombre', 'O'), ('ganó', 'O'), ('con', 'O'), ('el', 'O'), ('Real', 'B-ORG'), ('Madrid', 'I-ORG'), ('de', 'O'), ('España', 'B-ORG'), ('en', 'O'), ('Alemania', 'B-ORG'), ('con', 'O'), ('Roler', 'B-ORG')]


In [11]:
labels = list(crf.classes_)
labels.remove('O')
labels

['B-LOC', 'B-ORG', 'B-PER', 'I-PER', 'B-MISC', 'I-ORG', 'I-LOC', 'I-MISC']

In [12]:
y_pred = crf.predict(X_test)
metrics.flat_f1_score(y_test, y_pred, 
                      average='weighted', labels=labels)

0.7964686316443963

Resultados de precisión por etiquetas

In [13]:
# group B and I results
sorted_labels = sorted(
    labels, 
    key=lambda name: (name[1:], name[0])
)
print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

              precision    recall  f1-score   support

       B-LOC      0.810     0.784     0.797      1084
       I-LOC      0.690     0.637     0.662       325
      B-MISC      0.731     0.569     0.640       339
      I-MISC      0.699     0.589     0.639       557
       B-ORG      0.807     0.832     0.820      1400
       I-ORG      0.852     0.786     0.818      1104
       B-PER      0.850     0.884     0.867       735
       I-PER      0.893     0.943     0.917       634

   micro avg      0.813     0.787     0.799      6178
   macro avg      0.791     0.753     0.770      6178
weighted avg      0.809     0.787     0.796      6178

